# Importing packages

In [ ]:
# !pip install mne

In [ ]:
# !pip install pyxdf

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import mne
import pyxdf
import glob
import os
import matplotlib.pyplot as plt
from scipy.io import loadmat
import scipy
import sklearn
# ------------------------------------------------------------------------------------------
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda
from sklearn.model_selection import ShuffleSplit, cross_val_score, train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC

# ------------------------------------------------------------------------------------------
from mne.decoding import CSP
from mne import Epochs, pick_types
from mne.channels import make_standard_montage
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
from FBCSP_V4 import FBCSP_V4 as FBCSP 

In [ ]:
import warnings
warnings.filterwarnings('ignore') # to ignore warnings

In [ ]:
verbose = False                    # global variable to suppress output display of MNE functions
mne.set_log_level(verbose=verbose) # to suppress large info outputs

# Data Loading

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Anwar_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Anwar\ses-S002\eeg"
# Anwar_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Anwar\ses-S002\eeg\Remaining Data"
# Anwar_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Anwar\ses-S002\s_2_all"

# Anwar_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Anwar\s-00\eeg"
# Anwar_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Anwar\ses-S003\s3_00"
Anwar_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Anwar\s-00"
# sherif_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Sherif\s-00"
# sherif_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Sherif\ses-S002\eeg"
# sherif_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Sherif\ses-S003\eeg"
# magdy_data_path = "E:\BCI\A-Our Motor Data\magdy\sub-magdy\ses-S001\eeg"

folder_path = Anwar_data_path
files   = glob.glob(folder_path + '/*eeg.xdf')
# files   = glob.glob(folder_path)
len(files)     # if  return zero,then no file is loaded

In [ ]:
files

In [ ]:
# Filter the paths
filtered_paths = [path for path in files if ('Right Grasp' in path or 'Right Release' in path) and 
                                            ('B_Right Grasp' not in path and 'B_Right Release' not in path)]

# filtered_paths = [path for path in files if ('Right Grasp' in path or 'Right Release' in path or 'Baseline' in path) and 
#                                             ('B_Right Grasp' not in path and 'B_Right Release' not in path and 'B_Baseline' not in path)]

# Print the number of filtered paths and the paths themselves
print(f'Number of filtered paths: {len(filtered_paths)}')
filtered_paths

In [ ]:
all_files = filtered_paths
len(all_files)

# from XDF to MNE

In [ ]:
# read stream from xdf
streams, header = pyxdf.load_xdf(all_files[0])
# extract data
data = streams[0]["time_series"].T
# check that it is 9 channels
# assert data.shape[0] == 9 # 9 raw EEG channels

#get channels count
ch_count = int(streams[0]["info"]["channel_count"][0])
# extract channels names
# ch_names = []
# for i in range(ch_count):
#   ch_names.append(streams[0]["info"]["desc"][0]["channels"][0]["channel"][i]["label"][0])
# extract sampling ratw
fs = float(streams[0]["info"]["nominal_srate"][0])
# create info
# info = mne.create_info(ch_names, fs, "eeg")

In [ ]:
# extract channels names
ch_names = []
for i in range(ch_count):
  ch_name = streams[0]["info"]["desc"][0]["channels"][0]["channel"][i]["label"][0]
  # ch_name= ch_name.split("\n")[1]
  ch_name= ch_name.split("\n")[0]
  ch_names.append(ch_name)

In [ ]:
ch_names

In [ ]:
# cleaned_channel_names = [name.replace('[', '').replace(']', '') for name in ch_names]
# cleaned_channel_names

In [ ]:
crack_channel_names = ['C3', 'Cz', 'C4', 'Fz', 'F3', 'P4', 'Pz', 'P3', 'F4']

In [ ]:
# info = mne.create_info(crack_channel_names, fs, "eeg")
info = mne.create_info(ch_names, fs, "eeg")

In [ ]:
def extract_tasks_names(files):
  task_names = []
  for path in files:
      # Split the path to isolate the filename
      filename = path.split('/')[-1]
      # Extract the task name part
      task_name = filename.split('_')[-3]
      # Replace the hyphen with a space to match the desired format
      task_name = task_name.split('-')[-1]
      # Append the task name to the list
      task_names.append(task_name)
  return task_names

In [ ]:
task_names = extract_tasks_names(all_files)

task_names

In [ ]:
set(task_names)

In [ ]:
# event_id = {'Left_Grasp': 1, 'Right_Grasp': 2, 'Right_Release': 3, 'Baseline': 4}
event_id = {'Right Grasp': 1, 'Right Release': 2, 'Baseline': 3}
# Convert the list of task names to the list of numbers
task_numbers = [event_id[task] for task in task_names]
print(task_numbers)

In [ ]:
print(len(task_names))
# task_names

In [ ]:
# crop parametes
t_start = 1
t_end = 5
# t_end = 6

idx_start = int(fs*t_start)
idx_end = int(fs*t_end)

In [ ]:
all_data=[]
for fname in all_files:
  streams, header = pyxdf.load_xdf(fname)
  data = streams[0]["time_series"].T
  # crop data from second 1 to 6
  cropped_data = data[:,idx_start:idx_end+1]
  print(fname)
  all_data.append(cropped_data)

In [ ]:
len(all_data)
for data in all_data:
  print(data.shape)


In [ ]:
# Combine into a 3D array
combined_array = np.stack(all_data, axis=0)
print(combined_array.shape)

In [ ]:
eeg_data = combined_array

In [ ]:
epochs = mne.EpochsArray(eeg_data, info, verbose=verbose, tmin=0)

In [ ]:
# epochs.set_montage('standard_1020')
epochs.filter(1., None)
# epochs.apply_baseline(baseline=(-.250, 0)) # linear baseline correction

epochs.event_id = event_id
epochs.events[:,2] = task_numbers

In [ ]:
epochs

## Apply FBCSP

In [ ]:
verbose_clf = False # control output of FBCSP function
freqs_band = np.linspace(8, 32, 7) # filter bank choice
cv = 10
train_ratio = 0.75 # 75:25 for trian-valid split

In [ ]:
freqs_band

In [ ]:
epochs_list_train = epochs

In [ ]:
epochs.filter(7.0, 32.0)

## FBCSP 
The class must receive in input with the initialization a training set inside a dictionary. The keys of the dictionary must be the label of the two class and each element must be a numpy matrix of dimension "n. trials x n. channels x n.samples". The class must also receive the frequency sampling of the signal.

FBCSP function original has a built-in random splitting so I didn't do a manual splitting here

In [ ]:
data, labels = epochs.get_data(), epochs.events[:,-1]

In [ ]:
print(data.shape)
print(labels.shape)
labels

In [ ]:
data_dict = {'Right Grasp':  epochs['Right Grasp'].get_data(), 
            'Right Release': epochs['Right Release'].get_data()}


In [ ]:
epochs['Right Grasp'][0].get_data().shape

In [ ]:
epochs.get_data()[0].shape

In [ ]:
print(data_dict["Right Grasp"].shape)

In [ ]:
data_dict["Right Grasp"][0].shape

In [ ]:
fs = epochs.info['sfreq']
fs

In [ ]:
event_id = {'Right Grasp': 1, 'Right Release': 2, 'Baseline': 3}


In [ ]:
fbcsp_clf = FBCSP(data_dict, fs, freqs_band=freqs_band, classifier=lda(), train_ratio=train_ratio)

In [ ]:
data_matrix = fbcsp_clf.tmp_data_matrix
labels = fbcsp_clf.tmp_label
labels_dict = fbcsp_clf.tmp_label_dict
labels_dict = fbcsp_clf.n_features_for_classification

In [ ]:
labels

In [ ]:
X = data_matrix
y = labels

In [ ]:
print(data_matrix.shape)

## Normal Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, stratify=labels,test_size=0.4, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
print(y_train.shape)
print(y_train)
print(y_test.shape)
print(y_test)

In [ ]:
print(type(X_train))
print(type(y_train))

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import accuracy_score

lda_classifier = LDA()
lda_classifier.fit(X_train, y_train)

In [ ]:
# Step 7: Evaluate the Model
y_pred = lda_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
kappa_scorer = sklearn.metrics.make_scorer(sklearn.metrics.cohen_kappa_score) 
accuracy_scorer = sklearn.metrics.make_scorer(sklearn.metrics.accuracy_score)

In [ ]:
# Accuracy Score
train_score = accuracy_scorer(lda_classifier, X_train, y_train)
test_score = accuracy_scorer(lda_classifier, X_test, y_test)
print("Accuracy Score on Training set: ", train_score)
print("Accuracy Score on Test set: ", test_score)

In [ ]:
# Kappa Score
train_score = kappa_scorer(lda_classifier, X_train, y_train)
test_score = kappa_scorer(lda_classifier, X_test, y_test)
print("Kappa Score on Training set: ", train_score)
print("Kappa Score on Test set: ", test_score)
        

### All Models

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Dictionary to hold models and their names
models = {
    'SVM': SVC(kernel='rbf', C=1, gamma='scale'),
    'Linear SVC': LinearSVC(max_iter=10000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'k-NN': KNeighborsClassifier(n_neighbors=5),
    'Logistic Regression': LogisticRegression(solver='lbfgs', max_iter=1000),
    'LDA': LinearDiscriminantAnalysis(),
    'Naive Bayes': GaussianNB(),
    'Extra Trees': ExtraTreesClassifier(n_estimators=100, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'MLP Classifier': MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42),
}

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, stratify=labels,test_size=0.3, random_state=42)
print("Train Shape: ", X_train.shape)
print("Test Shape: ", X_test.shape)

In [ ]:
# Dictionary to store the trained models and their accuracies
# trained_models = {}
# model_accuracies = {}

# # Train and evaluate each model
# for name, model in models.items():
#     model.fit(X_train, y_train)  # Train the model
#     y_pred = model.predict(X_test)  # Predict on test data
#     accuracy = accuracy_score(y_test, y_pred)  # Calculate accuracy
#     print(f'{name} Accuracy: {accuracy * 100:.2f}%')
    
#     # Save the trained model to the dictionary
#     trained_models[name] = model
#     # Save the accuracy to the dictionary
#     model_accuracies[name] = accuracy * 100


In [ ]:
# Transform the labels to start from 0
label_encoder = LabelEncoder()
y_transformed = label_encoder.fit_transform(y)
# Split the data into training and testing sets
X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = train_test_split(X, y_transformed, shuffle=True, stratify=labels,test_size=0.3, random_state=42)
# Initialize the XGBoost classifier
xgb_model = XGBClassifier(
    n_estimators=100,         # Number of trees in the ensemble
    learning_rate=0.1,        # Step size shrinkage used to prevent overfitting
    max_depth=3,              # Maximum depth of a tree
    use_label_encoder=False,  # Disable label encoder to avoid deprecation warning
    eval_metric='logloss',    # Evaluation metric
    random_state=42           # Seed for reproducibility
)
# Train the XGBoost model
xgb_model.fit(X_train_encoded, y_train_encoded)
# Predict on the test set
y_pred_transformed = xgb_model.predict(X_test_encoded)
# Transform predictions back to original labels
y_pred = label_encoder.inverse_transform(y_pred_transformed)
# Calculate accuracy
accuracy = accuracy_score(label_encoder.inverse_transform(y_test_encoded), y_pred)
# Print the accuracy
print(f'XGBoost Accuracy: {accuracy * 100:.2f}%')
# model_accuracies["XGBoost"] = accuracy * 100


In [ ]:
# # Find the model with the highest accuracy
# best_model_name = max(model_accuracies, key=model_accuracies.get)
# max_accuracy = model_accuracies[best_model_name]

# # Print the model name and its maximum accuracy
# print(f'Best Model: {best_model_name} with Accuracy: {max_accuracy:.2f}%')

## StratifiedKFold Cross Validation

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
X = data_matrix
y = labels

In [ ]:
print(X.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, stratify=labels,test_size=0.3, random_state=42)
print("Train Shape: ", X_train.shape)
print("Test Shape: ", X_test.shape)

In [ ]:
# # Define the LDA classifier
# cv_lda_classifier = LDA()

# # Use StratifiedKFold to maintain class distribution in each fold
# kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=43)

# # Perform cross-validation and compute accuracy for each fold
# cv_scores = cross_val_score(cv_lda_classifier, X, y, cv=kf, scoring='accuracy')
# # Print cross-validation scores
# print(f'Cross-validation scores for each fold: {cv_scores}')
# print(f'Average cross-validation accuracy: {cv_scores.mean():.2f} ± {cv_scores.std():.2f}')

In [ ]:
# Dictionary to store the trained models and their accuracies
trained_models = {}
model_accuracies = {}
cv_results = {}
kappa_results = {}

# Variable to track the best cross-validation score and corresponding model name
best_cv_score = 0
best_cv_model_name = ""

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)  # Train the model
    y_pred = model.predict(X_test)  # Predict on test data
    accuracy = accuracy_score(y_test, y_pred)  # Calculate accuracy
    print(f'{name} Accuracy: {accuracy * 100:.2f}%')

    # Kappa Score
    kappa_score = kappa_scorer(lda_classifier, X_test, y_test)
    print("Kappa Score on Test set: ", test_score)
    
    # Save the trained model to the dictionary
    trained_models[name] = model
    # Save the accuracy to the dictionary
    # model_accuracies[name] = accuracy * 100
    model_accuracies[name] = accuracy 
    # kappa_results[name] = kappa_score * 100
    kappa_results[name] = kappa_score 
    
    # Perform Stratified K-Fold cross-validation
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=43)
    cv_scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
    cv_results[name] = cv_scores

    # Check if this model has the best cross-validation score
    if cv_scores.mean() > best_cv_score:
        best_cv_score = cv_scores.mean()
        best_cv_model_name = name

    # Print cross-validation results
    print(f'Cross-validation scores for {name}: {cv_scores}')
    print(f'Average cross-validation accuracy for {name}: {cv_scores.mean():.2f} ± {cv_scores.std():.2f}')

    print("--"*50)

In [ ]:
# Print the model with the highest average cross-validation score
print(f'Best Model based on cross-validation: {best_cv_model_name} with Average CV Accuracy: {best_cv_score:.2f}')

In [ ]:
# Find the model with the highest test set accuracy
best_model_name = max(model_accuracies, key=model_accuracies.get)
max_accuracy = model_accuracies[best_model_name]
print(f'Best Model based on test accuracy: {best_model_name} with Accuracy: {max_accuracy:.2f}%')

In [ ]:
# Find the model with the highest test set accuracy
best_kappa_model_name = max(kappa_results, key=kappa_results.get)
max_kappa = kappa_results[best_kappa_model_name]
print(f'Best Model based on test Kappa : {best_kappa_model_name} with Kappa: {max_kappa:.2f}%')

In [ ]:
# # Optionally, you can save the best model for later use
# best_model = trained_models[best_model_name]
# joblib.dump(best_model, f'{best_model_name}_model.joblib')

# To load and use the best model later
# loaded_best_model = joblib.load(f'{best_model_name}_model.joblib')
# predictions_transformed = loaded_best_model.predict(new_data)  # Replace 'new_data' with your actual data
# predictions = label_encoder.inverse_transform(predictions_transformed)

## Other Cross Validation

In [ ]:
# #KFold
# from sklearn.model_selection import KFold, cross_val_score
# cv = KFold(n_splits=5, shuffle=True)
# # cv_split = cv.split(X)
# cv_scores = cross_val_score(cv_lda_classifier, X,y, cv=cv, n_jobs=None)
# # Print cross-validation scores
# print(f'Cross-validation scores for each fold: {cv_scores}')
# print(f'Average cross-validation accuracy: {cv_scores.mean():.2f} ± {cv_scores.std():.2f}')

In [ ]:
# #ShuffleSplit
# cv = ShuffleSplit(5, test_size=0.2, random_state=42)
# # cv_split = cv.split(X)
# cv_scores = cross_val_score(cv_lda_classifier, X,y, cv=cv, n_jobs=None)
# # Print cross-validation scores
# print(f'Cross-validation scores for each fold: {cv_scores}')
# print(f'Average cross-validation accuracy: {cv_scores.mean():.2f} ± {cv_scores.std():.2f}')